In [1]:
%load_ext autoreload
%autoreload 2

In [23]:
import numpy as np
import pandas as pd

from functions import preprocess, train_default_catboost

In [63]:
df_train, df_test = preprocess('./data/train.csv'), preprocess('./data/test.csv')

In [64]:
df_train.head()

,Код_группы,Год_Поступления,Основания,КодФакультета,СрБаллАттестата,Статус,Код1,Код2,Код3,Муж,Год_Рождения,Возраст_Поступления,Перерыв,ПолнаяСемья,Учеба,МестоЖит
ID,,,,,,,,,,,,,,,,
72716,16019,2015,оо,41,85.88,3,16,01,019,0,1998,17,1,False,ш,брн
63306,14895,2010,цн,28,67.00,4,14,89,895,1,1992,18,0,True,н,брн
82469,20103,2017,дн,41,82.66,4,20,10,103,0,1999,18,0,True,ш,алт
81661,19838,2017,бн,35,74.00,4,19,83,838,0,1994,23,0,True,у,брн
81509,19803,2017,бн,28,53.00,4,19,80,803,0,1995,22,0,True,у,брн


In [65]:
df_train.dtypes

Код_группы             category
Год_Поступления           int64
Основания              category
КодФакультета          category
СрБаллАттестата         float64
Статус                    int64
Код1                   category
Код2                   category
Код3                   category
Муж                       int32
Год_Рождения              int64
Возраст_Поступления       int64
Перерыв                   int32
ПолнаяСемья                bool
Учеба                  category
МестоЖит               category
dtype: object

In [66]:
X, y = df_train.drop(['Статус'], axis=1, errors='ignore'), df_train['Статус'].replace({-1: 0, 3: 1, 4: 2})
X_test = df_test.drop(['Статус'], axis=1, errors='ignore')

In [67]:
clf = train_default_catboost(X, y, X_test, 'results/catboost_default.csv')

Train len: 10867, val len: 2717, test len: 6691
F1 train: 0.881687588455572 F1 test: 0.7817380711604728
{'Код_группы': 16.86698033907099, 'Год_Поступления': 5.431756046754685, 'Основания': 10.828736623066586, 'КодФакультета': 8.738217985954343, 'СрБаллАттестата': 5.355742232357824, 'Код1': 15.06653175817173, 'Код2': 11.897693766033004, 'Код3': 6.06430355963682, 'Муж': 1.6963455053416714, 'Год_Рождения': 3.425357935511717, 'Возраст_Поступления': 3.221703143214866, 'Перерыв': 1.9364246044997169, 'ПолнаяСемья': 0.7460527684362187, 'Учеба': 5.405085066068086, 'МестоЖит': 3.319068665881889}
